In [459]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np

In [460]:
dfcases2018 = pd.read_csv('/Users/ashnadua/Desktop/PrecogTask/csv/cases/cases/cases_2018.csv', dtype={
    'ddl_case_id': str
})

In [461]:
dfcases2018 = dfcases2018.drop(['state_code', 'dist_code', 'court_no', 'cino', 'female_defendant', 'female_petitioner', 'female_adv_def', 'female_adv_pet', 'type_name', 'purpose_name', 'disp_name', 'date_of_filing', 'date_of_decision', 'date_first_list', 'date_last_list', 'date_next_list', 'judge_position'], axis=1)

In [462]:
caseYears = dict(zip(dfcases2018['ddl_case_id'], dfcases2018['year']))

In [463]:
dfActs = pd.read_csv('/Users/ashnadua/Desktop/PrecogTask/csv/acts_sections.csv', dtype={
    'ddl_case_id': str,
    'act': 'Int64',
    'section': 'Int64',
    'bailable_ipc': bytes,
    'number_sections_ipc': bytes,
    'criminal': int
})

In [464]:
dfActs = dfActs.drop(['act', 'section', 'bailable_ipc', 'number_sections_ipc'], axis=1)

In [465]:
dfActs['year'] = dfActs['ddl_case_id'].map(caseYears)

In [466]:
dfActs = dfActs.dropna()

In [467]:
dfActs = dfActs.drop(['year'], axis=1)

In [468]:
criminalCheck = dict(zip(dfActs['ddl_case_id'], dfActs['criminal']))

In [469]:
dfJudge = pd.read_csv('/Users/ashnadua/Desktop/PrecogTask/csv/judges_clean.csv')

In [470]:
gender = dict(zip(dfJudge['ddl_judge_id'], dfJudge['female_judge']))

In [471]:
dfCaseName = pd.read_csv('/Users/ashnadua/Desktop/PrecogTask/csv/keys/keys/judge_case_merge_key.csv', dtype={
    'ddl_case_id': str,
    'ddl_filing_judge_id': 'Int64',
    'ddl_decision_judge_id': 'Int64'
})

In [472]:
dfCaseName2018 = dfCaseName.copy()

In [473]:
dfCaseName2018.rename(columns = {'ddl_case_id':'case_id'}, inplace = True)

In [474]:
dfCaseName2018['filing_gender'] = dfCaseName2018['ddl_filing_judge_id'].map(gender)

In [475]:
dfCaseName2018['decision_gender'] = dfCaseName2018['ddl_decision_judge_id'].map(gender)

In [476]:
dfCaseName2018 = dfCaseName2018.dropna()

In [491]:
pd.set_option('mode.chained_assignment',None)

In [492]:
dfcases2018.reset_index(inplace=True, drop=True)

In [493]:
dfCaseName2018['criminal'] = dfCaseName2018['case_id'].map(criminalCheck)

In [494]:
dfCaseName2018 = dfCaseName2018.dropna()

In [495]:
count2018 = dfCaseName2018.filing_gender.loc[(dfCaseName2018['criminal'] == 1)].value_counts().to_frame()

In [496]:
countCivil2018 = dfCaseName2018.filing_gender.loc[(dfCaseName2018['criminal'] == 0)].value_counts().to_frame()

In [497]:
count2018 = count2018.reset_index()
countCivil2018 = countCivil2018.reset_index()

In [498]:
count2018.at[0,'index']='Male'
count2018.at[1,'index']='Female'
count2018.at[2,'index']='Unclear'

In [499]:
countCivil2018.at[0,'index']='Male'
countCivil2018.at[1,'index']='Female'
countCivil2018.at[2,'index']='Unclear'

In [508]:
colors = ['#195190FF', '#FF6E40', '#FFC13B'] 
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]],
)

fig.add_trace(go.Pie(values=count2018['filing_gender'], labels=count2018['index'], pull=[0.05, 0.05, 0.1], marker_colors=colors, title="Criminal Cases"),
              row=1, col=1)

fig.add_trace(go.Pie(values=countCivil2018['filing_gender'], labels=countCivil2018['index'], pull=[0.05, 0.05, 0.1], marker_colors=colors, title="Civil Cases", rotation=100),
              row=1, col=2)

fig.update_layout(height=500, showlegend=True, title=go.layout.Title(
        text="Gender Bias 2017",
        xref="paper",
        x=0.5
        ))



fig.show()
